In [1]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from random import randrange
import torch.nn.functional as F


plt.ion()   # interactive mode

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


dat_race = pd.read_csv('dnd_race.csv')
dat_class = pd.read_csv('dnd_class.csv')

dat_race['image_path'] = dat_race['label']+'/'+dat_race['image_number']
dat_class['image_path'] = dat_class['label']+'/'+dat_class['image_number']

dat_race = dat_race.rename(index=str, columns={"label": "race"})
dat_class = dat_class.rename(index=str, columns={"label": "class"})
dat_class.head(1)


X_race = dat_race['image_path']
X_class = dat_class['image_path']

y_race = pd.get_dummies(dat_race['race'])
y_class = pd.get_dummies(dat_class['class'])


In [3]:
dat_class['class'].value_counts()

rogue      148
paladin    147
wizard     119
Name: class, dtype: int64

In [4]:
148/dat_class.shape[0]

0.357487922705314

In [5]:
dat_race.race.value_counts()

elf           188
gnome         122
dragonborn    120
Name: race, dtype: int64

In [6]:
188/dat_race.shape[0]

0.4372093023255814

In [7]:
X_race_train, X_race_test, y_race_train, y_race_test = train_test_split(X_race, y_race, test_size=0.20, random_state=42)
X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(X_class, y_class, test_size=0.20, random_state=42)


In [8]:
X_race_train = X_race_train.values.tolist()
X_race_test = X_race_test.values.tolist()


X_class_train = X_class_train.values.tolist()
X_class_test = X_class_test.values.tolist()

race_shape = y_race_train.shape[1]
class_shape = y_class_train.shape[1]


y_race_train = y_race_train.values.tolist()
y_race_test = y_race_test.values.tolist()

y_class_train = y_class_train.values.tolist()
y_class_test = y_class_test.values.tolist()

In [ ]:
len(y_race_train)

344

In [ ]:
len(y_class_train)

331

In [ ]:
#for i in dataset_style:
#    print(i,dat[i].sum(),dat[i].sum()/dat.shape[0])

In [ ]:
#for i in dataset_rest:
#    print(i,dat[i].sum(),dat[i].sum()/dat.shape[0])

In [ ]:
import random
flip = random.randint(0, 1)
flip

1

In [ ]:
class fgo_dataset(Dataset):
    def __init__(self,king_of_lists, transform=None):
        """
        gender_img, style_img, gender_pred,style_pred 
        
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.king_of_lists = king_of_lists
        self.transform = transform
        
    def __getitem__(self,index):
        
        #random_index = randrange(len(self.king_of_lists[index]))
        
        img1 = Image.open('images/'+self.king_of_lists[0][index])
        img1 = img1.convert('RGB')
        
        #img2 = Image.open('images/'+self.king_of_lists[1][index])
        #img2 = img2.convert('RGB')
        
        
        gender = self.king_of_lists[1][index] # gender
        #color = self.king_of_lists[3][index] # region
        
        if self.transform is not None:
            img1 = self.transform(img1)

        list_of_labels = [torch.from_numpy(np.array(gender))]
       
        return img1, list_of_labels[0]
    
    def __len__(self):
        return len(self.king_of_lists[0])

In [ ]:
batch_size = 8
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256,256)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_lists = [X_race_train, y_race_train]
test_lists = [X_race_test, y_race_test ]

training_dataset = fgo_dataset(king_of_lists = train_lists,
                               transform = data_transforms['train'] )

test_dataset = fgo_dataset(king_of_lists = test_lists,
                           transform = data_transforms['val'] )

dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
                   }
dataset_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}


train_class_lists = [X_class_train, y_class_train]
test_class_lists = [X_class_test, y_class_test ]

training_style_dataset = fgo_dataset(king_of_lists = train_class_lists,
                               transform = data_transforms['train'] )

test_style_dataset = fgo_dataset(king_of_lists = test_class_lists,
                           transform = data_transforms['val'] )

dataloaders_style_dict = {'train': torch.utils.data.DataLoader(training_style_dataset, batch_size=batch_size,drop_last =True, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_style_dataset, batch_size=batch_size,drop_last =True, shuffle=True, num_workers=0)
                   }
dataloaders_style_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
print(len(dataloaders_style_dict['train']),len(dataloaders_style_dict['val']))

41 10


In [ ]:
print(len(dataloaders_dict['train']),len(dataloaders_dict['val']))

43 11


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders_dict[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #_, preds = torch.max(outputs, 1)
                    
                    loss = criterion(outputs, torch.max(labels.float(), 1)[1]).to(device)
                    #loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects +=torch.sum(torch.max(outputs, 1)[1] == torch.max(labels, 1)[1])
                #running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(best_model_wts, './models/dnd_resnet_50.pth')
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 3)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.00001,weight_decay = .01)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------


D:\Users\micha\Anaconda3\envs\tensorflow_env\lib\site-packages\PIL\Image.py:965: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


train Loss: 1.1190 Acc: 0.3576
val Loss: 1.0327 Acc: 0.5233

Epoch 1/24
----------
train Loss: 0.9740 Acc: 0.5843
val Loss: 0.9094 Acc: 0.7326

Epoch 2/24
----------
train Loss: 0.8695 Acc: 0.7064
val Loss: 0.8012 Acc: 0.7442

Epoch 3/24
----------
train Loss: 0.7860 Acc: 0.7238
val Loss: 0.7141 Acc: 0.7326

Epoch 4/24
----------
train Loss: 0.6995 Acc: 0.7733
val Loss: 0.6348 Acc: 0.7674

Epoch 5/24
----------
train Loss: 0.6342 Acc: 0.7936
val Loss: 0.6146 Acc: 0.7791

Epoch 6/24
----------
train Loss: 0.5853 Acc: 0.7849
val Loss: 0.6076 Acc: 0.7791

Epoch 7/24
----------
train Loss: 0.5611 Acc: 0.8081
val Loss: 0.5981 Acc: 0.7791

Epoch 8/24
----------
train Loss: 0.6060 Acc: 0.7616
val Loss: 0.6026 Acc: 0.7791

Epoch 9/24
----------
train Loss: 0.5599 Acc: 0.8285
val Loss: 0.6081 Acc: 0.7674

Epoch 10/24
----------
train Loss: 0.5364 Acc: 0.8256
val Loss: 0.5873 Acc: 0.7907

Epoch 11/24
----------
train Loss: 0.5496 Acc: 0.8459
val Loss: 0.5929 Acc: 0.7791

Epoch 12/24
----------
t

# Eval
 